# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [18]:
# import numpy and pandas
import numpy as np
import pandas as pd

import seaborn as sns
from scipy.stats import ttest_1samp, ttest_rel, ttest_ind
from scipy.stats import f_oneway

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents.

In [2]:
# Your code here:

salaries = pd.read_csv('Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examine the `salaries` dataset using the `head` function below.

In [3]:
# Your code here:

salaries.head(20)

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86
5,"ABBASI, CHRISTOPHER",STAFF ASST TO THE ALDERMAN,CITY COUNCIL,F,Salary,NaN,50436.0,NaN
6,"ABBATACOLA, ROBERT J",ELECTRICAL MECHANIC,AVIATION,F,Hourly,40.0,NaN,46.10
7,"ABBATE, JOSEPH L",POOL MOTOR TRUCK DRIVER,STREETS & SAN,F,Hourly,40.0,NaN,35.60
8,"ABBATEMARCO, JAMES J",FIRE ENGINEER-EMT,FIRE,F,Salary,NaN,103350.0,NaN
9,"ABBATE, TERRY M",POLICE OFFICER,POLICE,F,Salary,NaN,93354.0,NaN


In [4]:
salaries.shape

(33183, 8)

In [5]:
salaries.columns

Index(['Name', 'Job Titles', 'Department', 'Full or Part-Time',
       'Salary or Hourly', 'Typical Hours', 'Annual Salary', 'Hourly Rate'],
      dtype='object')

We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [6]:
# Your code here:

dsnull = salaries.isnull().sum()
print(dsnull)

Name                     0
Job Titles               0
Department               0
Full or Part-Time        0
Salary or Hourly         0
Typical Hours        25161
Annual Salary         8022
Hourly Rate          25161
dtype: int64


Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [7]:
# Your code here:

salaried = [i for i in salaries['Salary or Hourly'] if i == 'Salary']
len(salaried)

25161

In [8]:
hourly = [i for i in salaries['Salary or Hourly'] if i == 'Hourly']
len(hourly)

8022

What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [9]:
# Your code here:
salaries.Department.unique()

array(['POLICE', 'GENERAL SERVICES', 'WATER MGMNT', 'OEMC',
       'CITY COUNCIL', 'AVIATION', 'STREETS & SAN', 'FIRE',
       'FAMILY & SUPPORT', 'PUBLIC LIBRARY', 'TRANSPORTN',
       "MAYOR'S OFFICE", 'HEALTH', 'BUSINESS AFFAIRS', 'LAW', 'FINANCE',
       'CULTURAL AFFAIRS', 'COMMUNITY DEVELOPMENT', 'PROCUREMENT',
       'BUILDINGS', 'ANIMAL CONTRL', 'CITY CLERK', 'BOARD OF ELECTION',
       'DISABILITIES', 'HUMAN RESOURCES', 'DoIT', 'BUDGET & MGMT',
       'TREASURER', 'INSPECTOR GEN', 'HUMAN RELATIONS', 'COPA',
       'BOARD OF ETHICS', 'POLICE BOARD', 'ADMIN HEARNG',
       'LICENSE APPL COMM'], dtype=object)

In [10]:
salaries.groupby(['Department']).size()

Department
ADMIN HEARNG                39
ANIMAL CONTRL               81
AVIATION                  1629
BOARD OF ELECTION          107
BOARD OF ETHICS              8
BUDGET & MGMT               46
BUILDINGS                  269
BUSINESS AFFAIRS           171
CITY CLERK                  84
CITY COUNCIL               411
COMMUNITY DEVELOPMENT      207
COPA                       116
CULTURAL AFFAIRS            65
DISABILITIES                28
DoIT                        99
FAMILY & SUPPORT           615
FINANCE                    560
FIRE                      4641
GENERAL SERVICES           980
HEALTH                     488
HUMAN RELATIONS             16
HUMAN RESOURCES             79
INSPECTOR GEN               87
LAW                        407
LICENSE APPL COMM            1
MAYOR'S OFFICE              85
OEMC                      2102
POLICE                   13414
POLICE BOARD                 2
PROCUREMENT                 92
PUBLIC LIBRARY            1015
STREETS & SAN             21

# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [11]:
# Your code here: H0 -> the hourly wage == $30/hr  -----> H1 -> the hourly wage != $30/hr
from scipy.stats import ttest_1samp

ttest_1samp(salaries['Hourly Rate'], 30, nan_policy='omit')

# other option = filter salaries['Hourly Rate'], for zeros

Ttest_1sampResult(statistic=20.619805785494147, pvalue=4.323024048614701e-92)

We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [12]:
# Your code here: salaries in the police force H0: salary mean = 86000 H1: salary mean != 86000

ttest_1samp(salaries['Annual Salary'], 86000, nan_policy='omit')

Ttest_1sampResult(statistic=5.932870515690814, pvalue=3.015758519929416e-09)

Using the `crosstab` function, find the department that has the most hourly workers. 

In [13]:
# Your code here:

tab = pd.crosstab(salaries['Department'],salaries['Salary or Hourly'])
tab.sort_values(['Hourly'], ascending=False)

Salary or Hourly,Hourly,Salary
Department,,
STREETS & SAN,1862,336
WATER MGMNT,1513,366
OEMC,1273,829
AVIATION,1082,547
GENERAL SERVICES,765,215
TRANSPORTN,725,415
PUBLIC LIBRARY,299,716
FAMILY & SUPPORT,287,328
CITY COUNCIL,64,347


The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [14]:
# Your code here: H0; salary == 35    H1: salary < 35

ttest_1samp(salaries[salaries['Department']=='STREETS & SAN']['Hourly Rate'], 35, nan_policy='omit')

Ttest_1sampResult(statistic=-9.567447887848152, pvalue=3.337853056471074e-21)

# Challenge 3: To practice - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [30]:
# Your code here: 95% confidence interval for the mean hourly wage of all hourly workers
from scipy.stats import norm
from scipy import stats

sample = salaries['Hourly Rate']
sample = sample.dropna(how='all')

ci = 0.95
mean = sample.mean()
n = len(sample)-1
ste = stats.sem(sample)

Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [31]:
# Your code here:

interval = stats.t.interval(ci, n, loc = mean, scale = ste)
interval

(32.52345834488529, 33.05365708767727)

# Bonus Challenge - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [ ]:
# Your code here:

